In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Concatenate,Dropout,Bidirectional,Attention,Flatten,Reshape
from tensorflow.keras.optimizers import Adam

import re
import nltk
from nltk.corpus import stopwords
import tensorflow_hub as hub
import tensorflow_text as text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
 !pip install transformers
 !pip install tensorflow_text

In [3]:
data_train=pd.read_csv('/kaggle/input/twitter-data/twitter_training.csv')
data_test=pd.read_csv('/kaggle/input/twitter-data/twitter_validation.csv')

In [4]:
data_train.columns=['no1','no2','sentiment','text']
data_train.drop('no1',axis=1,inplace=True)
data_train.drop('no2',axis=1,inplace=True)
data_train.head()

,sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [5]:
data_test.columns=['no1','no2','sentiment','text']
data_test.drop('no1',axis=1,inplace=True)
data_test.drop('no2',axis=1,inplace=True)
data_test

,sentiment,text
0,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,Negative,@Microsoft Why do I pay for WORD when it funct...
2,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,Neutral,Now the President is slapping Americans in the...
4,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...
994,Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,Positive,Today sucked so it’s time to drink wine n play...
997,Positive,Bought a fraction of Microsoft today. Small wins.


In [6]:
data_train.dropna(inplace=True)
data_test.dropna(inplace=True)

In [7]:
data_train['sentiment']=data_train['sentiment'].map({'Positive':0,'Negative':2,'Neutral':1,'Irrelevant':3})
data_test['sentiment']=data_test['sentiment'].map({'Positive':0,'Negative':2,'Neutral':1,'Irrelevant':3})


In [8]:
x_train_sentiment=np.array(data_train.iloc[:,0])
x_train_sentiment=tf.one_hot(x_train_sentiment, 4)
x_train_sentiment

<tf.Tensor: shape=(73995, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>

In [9]:
x_test_sentiment=np.array(data_test.iloc[:,0])
x_test_sentiment=tf.one_hot(x_test_sentiment, 4)
x_test_sentiment

<tf.Tensor: shape=(999, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [12]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [13]:
x_train_text = []
sentences = list(data_train['text'])
for sen in sentences:
    x_train_text.append(preprocess_text(sen))


x_test_text = []
sentences = list(data_test['text'])
for sen in sentences:
    x_test_text.append(preprocess_text(sen))

In [14]:
x_train=x_train_text
y_train=x_train_sentiment
x_test=x_test_text
y_test=x_test_sentiment

In [15]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [16]:
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [17]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2",trainable=True)


In [18]:
# text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string),
#                ...] # This SavedModel accepts up to 2 text inputs.
# tokenize = hub.KerasLayer(preprocessor.tokenize)
# tokenized_inputs = [tokenize(segment) for segment in text_inputs]

# # Step 2 (optional): modify tokenized inputs.
# pass

# # Step 3: pack input sequences for the Transformer encoder.
# seq_length = 32 # Your choice here.
# bert_pack_inputs = hub.KerasLayer(
#     preprocessor.bert_pack_inputs,
#     arguments=dict(seq_length=seq_length))  # Optional argument.
# encoder_inputs = bert_pack_inputs(tokenized_inputs)

In [20]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# outputs=Reshape((256,1))(outputs["pooled_output"])

# CNN layer
cnn_layer = Conv1D(64,3,padding="same", activation='relu')(outputs["sequence_output"])
cnn_layer=Dropout(0.1)(cnn_layer)
cnn_layer = MaxPooling1D()(cnn_layer)

# CNN layer
cnn_layer2 = Conv1D(128, 5,padding="same",activation='relu')(cnn_layer)
# cnn_layer2 = MaxPooling1D()(cnn_layer2)
cnn_layer2=Dropout(0.1)(cnn_layer2)
#CNN layer
# num_filters = 256
# kernel_size = 7
cnn_layer2 = Conv1D(256, 7, activation='relu',padding="same")(cnn_layer2)
# cnn_layer2=Dropout(0.2)(cnn_layer2)
# cnn_layer2 = MaxPooling1D()(cnn_layer2)
# LSTM layer

lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(cnn_layer2)
lstm_layer = Bidirectional(LSTM(128))(lstm_layer)
#attention_layer=attention()(lstm_layer)
# lstm_layer = Attention()([lstm_layer, lstm_layer])
# lstm_layer=Flatten()(lstm_layer)

# Neural network layers
# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(4, activation='sigmoid', name="output")(lstm_layer)


# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
x_train=np.array(x_train)
x_test=np.array(x_test)


In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [24]:
model.fit(x_train, y_train, batch_size=256, epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
290/290 [==============================] - 803s 3s/step - loss: 0.3981 - accuracy: 0.6179 - val_loss: 0.2043 - val_accuracy: 0.8458
Epoch 2/5
290/290 [==============================] - 752s 3s/step - loss: 0.2143 - accuracy: 0.8213 - val_loss: 0.1468 - val_accuracy: 0.9049
Epoch 3/5
290/290 [==============================] - 750s 3s/step - loss: 0.1469 - accuracy: 0.8772 - val_loss: 0.1055 - val_accuracy: 0.9279
Epoch 4/5
290/290 [==============================] - 752s 3s/step - loss: 0.1178 - accuracy: 0.8999 - val_loss: 0.1073 - val_accuracy: 0.9339
Epoch 5/5
290/290 [==============================] - 747s 3s/step - loss: 0.0969 - accuracy: 0.9159 - val_loss: 0.1101 - val_accuracy: 0.9379


In [ ]:
# model.fit(x_train[:147990], y_train[:147990], batch_size=256, epochs=5,validation_data=(x_test[250000:295980],y_test[250000:295980]))

In [25]:
model.fit(x_train, y_train, batch_size=256, epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
290/290 [==============================] - 748s 3s/step - loss: 0.0871 - accuracy: 0.9242 - val_loss: 0.1336 - val_accuracy: 0.9359
Epoch 2/5
290/290 [==============================] - 740s 3s/step - loss: 0.0798 - accuracy: 0.9292 - val_loss: 0.1441 - val_accuracy: 0.9399
Epoch 3/5
290/290 [==============================] - 740s 3s/step - loss: 0.0833 - accuracy: 0.9280 - val_loss: 0.1232 - val_accuracy: 0.9369
Epoch 4/5
290/290 [==============================] - 736s 3s/step - loss: 0.0722 - accuracy: 0.9351 - val_loss: 0.1428 - val_accuracy: 0.9369
Epoch 5/5
290/290 [==============================] - 748s 3s/step - loss: 0.0655 - accuracy: 0.9413 - val_loss: 0.1239 - val_accuracy: 0.9409


In [30]:
y_test_pred=model.predict(x_test)
model.evaluate(x_test,y_test)

32/32 [==============================] - 3s 90ms/step - loss: 0.1239 - accuracy: 0.9409


[0.12393751740455627, 0.9409409165382385]

In [31]:
y_test_pred

array([[1.3953718e-04, 9.9942744e-01, 1.5714621e-04, 2.2881648e-04],
       [4.5077555e-04, 9.6667599e-04, 9.9832088e-01, 2.9120455e-04],
       [4.5065180e-04, 9.3818083e-04, 9.9838078e-01, 2.7919016e-04],
       ...,
       [9.9295396e-01, 1.0515202e-03, 1.7927173e-03, 4.0300693e-03],
       [9.9717772e-01, 4.9529603e-04, 6.6079409e-04, 1.1990430e-03],
       [1.4608569e-04, 9.9939382e-01, 1.6233408e-04, 2.3515444e-04]],
      dtype=float32)

In [32]:
input_array=y_test_pred
binary_array = np.zeros_like(input_array)
max_indices = np.argmax(input_array, axis=1)
binary_array[np.arange(len(input_array)), max_indices] = 1
y_test_pred=binary_array
print(binary_array)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       277
           1       0.95      0.93      0.94       285
           2       0.93      0.97      0.95       266
           3       0.95      0.91      0.93       171

   micro avg       0.94      0.94      0.94       999
   macro avg       0.94      0.94      0.94       999
weighted avg       0.94      0.94      0.94       999
 samples avg       0.94      0.94      0.94       999

